In [14]:
import pandas as pd
import re

In [15]:
filename = 'october2020.csv'
df = pd.read_csv(f"datasets/{filename}")
df.drop(df.columns[0], axis=1, inplace=True)
df.dropna(subset=["post"], inplace=True)
df.set_index('link', inplace=True)
df.head()

,date,post,title
link,,,
/r/mechmarket/comments/jlwj7h/sgh_new_and_unbuilt_tx66_white_gmk_coral_gmk/,1.604205e+09,[TX66 Timestamp](https://imgur.com/a/XvaXEQD)\...,"[SG][H] New and Unbuilt TX66 White, GMK Coral,..."
/r/mechmarket/comments/jlwi4o/usca_h_mojo60_ewhite_dz60rgbansi_v2_hotswap_pcb/,1.604205e+09,[**Timestamp**](https://imgur.com/a/LCN5jst)\n...,"[US-CA] [H] Mojo60 ewhite, DZ60RGB-ANSI v2 Hot..."
/r/mechmarket/comments/jlvr38/usca_h_dsa_granite_text_modifiers_dsa_high/,1.604201e+09,Timestamps:\n\n[https://drive.google.com/file/...,"[US-CA] [H] DSA Granite Text Modifiers, DSA Hi..."
/r/mechmarket/comments/jlvjt9/uscah_gmk_fuyu_kuro_shiro_w_paypal_cash/,1.604200e+09,Stamp: https://imgur.com/a/qAbzDg3\n\nGMK Fuyu...,"[US-CA][H] GMK Fuyu, Kuro Shiro [W] Paypal, Cash"
/r/mechmarket/comments/jlvcg1/uscahbuilt_kbd75v2_nk65ee_blumen_and_smoke_gmk/,1.604199e+09,[TIMESTAMP](https://imgur.com/a/zwALQZ3)\n\nHa...,"[US-CA][H]built KBD75V2, NK65ee blumen and smo..."


In [16]:
df['post_lower'] = df['post'].str.lower()
df['title_lower'] = df['title'].str.lower()
only_gmk = df[df['post_lower'].str.contains('gmk') | df['title_lower'].str.contains('gmk')].copy()

only_gmk.shape, df.shape

((1181, 5), (1188, 5))

In [35]:
from collections import defaultdict

money_regex = r'([£€\$]\d+)|(\d+[£€\$])'
euro_to_usd = 1.2
pound_to_usd = 1.4

sets = ['base', 'nov', 'alpha', 'accent', 'bars', 'spacebar', 'light base', 'dark base', 'deskmat', 'rama', '40s', '40\'s', 'fourties', 'mods', 'extension', 'numpad']

sales_data = []

def get_category(products):
    has_base = any(['base' in p for p in products])
    if len(products) == 1 and has_base:
        return 'base'
    elif len(products) == 1:
        return 'single'
    elif len(products) > 1 and has_base:
        return'bundle'
    
    return 'other'

def match_product(row):
    s = row.post_lower.split("\n")
    for l in s:
        low = l.lower()
        if "gmk " in low and ('~~' in low or 'sold' in low):
            after_gmk = low.split("gmk ")[1]
            product_name = "gmk " + re.split(r"[^\w\+\.]", after_gmk)[0]

            if "gmk stab" in product_name or "gmk screw" in product_name:
                continue

            matches = re.split(money_regex, low)
            temp_data = {}

            if len(matches) > 1:
                for i in range(0, len(matches)-1, 3):
                    curr_price = int(matches[i+1][1:] if matches[i+1] else matches[i+2][:-1])
                    currency = matches[i+1][0] if matches[i+1] else matches[i+2][-1]
                    if currency == "€":
                        curr_price = round(curr_price * euro_to_usd)
                    if currency == "£":
                        curr_price = round(curr_price * pound_to_usd)

                    curr_str = matches[i]
                    kits = []
                    removeBase = False

                    for x in sets:
                        if x in curr_str:
                            if x == 'nov':
                                kits.append('novelties')
                            elif x == 'light base' or x == 'dark base':
                                removeBase = True
                                kits.insert(0, x)
                            elif x == 'bars' or x == 'spacebar':
                                if 'spacebars' not in kits:
                                    kits.append('spacebars')
                            elif x in ['40s', '40\'s', 'fourties']:
                                kits.append('40s')
                            else:
                                kits.append(x)

                    if i == 0 and not kits:
                        kits.append('base')

                    if removeBase and 'base' in kits:
                        kits.remove('base')
                    
                    if i == 0:
                        has_base = any(['base' in k for k in kits])
                        if not has_base:
                            for x in after_gmk.split(' '):
                                if any([se in x for se in sets]):
                                    break
                                        
                                if '+' in x or ',' in x:
                                    if x == 'olivia++':
                                        continue

                                    kits.insert(0, 'base')
                                    break

                        temp_data["products"] = kits
                        temp_data["str"] = curr_str
                        temp_data["price"] = curr_price

                    if temp_data['products']:
                        temp_data['category'] = get_category(temp_data['products'])
                    
                    if kits and i > 0:
                        sales_data.append([row[0], product_name, temp_data['products'], temp_data['price'], temp_data['category'], row.date])
                        temp_data["products"] = kits
                        temp_data["str"] = curr_str
                        temp_data["price"] = curr_price
                    else:
                        temp_data["price"] = min(temp_data["price"], curr_price)

                if temp_data['products']:
                        temp_data['category'] = get_category(temp_data['products'])

                sales_data.append([row[0], product_name, temp_data['products'], temp_data['price'], temp_data['category'], row.date])
                

for row in df.itertuples():
    match_product(row)

In [36]:
sales_df = pd.DataFrame(sales_data, columns=['link', 'product', 'sets', 'price', 'category', 'date'])
sales_df['date'] = pd.to_datetime(sales_df['date'], unit = 's')

import unicodedata
remove_accents = lambda text: unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
sales_df['product'] = sales_df['product'].apply(remove_accents)
sales_df['product'].replace(r'\W+$', '', regex=True, inplace=True)

sales_df.to_csv('sales/sales_' + filename)

In [28]:
sales_df.groupby('product')['link'].count().sort_values(ascending=False).to_csv('test.csv')